In [1]:
import pickle
import numpy as np
from sklearn.model_selection import train_test_split
import math

# TODO: Implement load the data here.
with open('data/train.p', 'rb') as f:
    data = pickle.load(f)
    
X_train, X_val, y_train, y_val = train_test_split(data['features'], data['labels'], random_state=0, test_size=0.33)

In [2]:
import pickle
import vgg16
from keras.preprocessing import image
import numpy as np
import cv2
from scipy.misc import imresize

with open('data/train.p', 'rb') as f:
    data = pickle.load(f)

Using TensorFlow backend.


In [3]:
n_train = len(X_train)
n_classes = len(set(y_train))
image_shape = X_train[0].shape

print("Number of training examples =", n_train)
print("Number of classes =", n_classes)
print("Image data shape =", image_shape)

Number of training examples = 26270
Number of classes = 43
Image data shape = (32, 32, 3)


In [4]:
from keras.utils import np_utils

# input image dimensions
img_rows, img_cols = 32, 32
img_channels = 3

# convert class vectors to binary class matrices
y_train = np_utils.to_categorical(y_train, n_classes)
y_val = np_utils.to_categorical(y_val, n_classes)

X_train = X_train.astype('float32')
X_val = X_val.astype('float32')
X_train /= 255
X_val /= 255


In [5]:
from keras.applications.vgg16 import VGG16
from keras.preprocessing import image
from keras.applications.vgg16 import preprocess_input
from keras.layers import Input, Flatten, Dense
from keras.models import Model
import numpy as np

#Get back the convolutional part of a VGG network trained on ImageNet
model_vgg16_conv = VGG16(weights='imagenet', include_top=False)
# model_vgg16_conv.summary()

#Create your own input format (here 3x200x200)
input = Input(shape=(32,32,3),name = 'image_input')

#Use the generated model
output_vgg16_conv = model_vgg16_conv(input)

#Add the fully-connected layers 
x = Flatten(name='flatten')(output_vgg16_conv)
x = Dense(4096, activation='relu', name='fc1')(x)
x = Dense(4096, activation='relu', name='fc2')(x)
x = Dense(43, activation='softmax', name='predictions')(x)

#Create your own model 
my_model = Model(input=input, output=x)

my_model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])


In [6]:
from keras.callbacks import ModelCheckpoint
from keras.preprocessing.image import ImageDataGenerator

datagen = ImageDataGenerator(
    rotation_range=3,
    width_shift_range=0.03,
    height_shift_range=0.03)

my_model.fit_generator(generator=datagen.flow(X_train, y_train, batch_size=32),
          samples_per_epoch=X_train.shape[0],
          nb_epoch=32,
          validation_data=(X_val, y_val),
          callbacks=[ModelCheckpoint('model.h5',save_best_only=True)])

Epoch 1/32
26270/26270 [==============================] - 2250s - loss: 15.2318 - acc: 0.0541 - val_loss: 15.2025 - val_acc: 0.0568
Epoch 2/32
26270/26270 [==============================] - 3011s - loss: 15.2438 - acc: 0.0542 - val_loss: 15.2025 - val_acc: 0.0568
Epoch 3/32
 5248/26270 [====>.........................] - ETA: 2217s - loss: 15.3595 - acc: 0.0471

KeyboardInterrupt: 